In [1]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Model
import numpy as np

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model = GPT2Model.from_pretrained("gpt2")


In [3]:
GloVe = {}
with open("glove.6B/glove.6B.100d.txt", "r", encoding="utf-8") as vector_file:
    for line in vector_file:
        line_content = line.split()
        word = line_content[0]
        # There's probably a better way to read strings into a FloatTensor
        word_vec = torch.from_numpy(np.asarray(line_content[1:], "float32"))
        GloVe[word] = word_vec

In [ ]:
# Play with GloVe embeddings


In [18]:
# new implementation, using GloVe vectors

# TODO: Custom Beam Search -- Keep n possibilities (beams) at each time
# then, accumulate a probability associated with each (normalize by length of generation)

prompt = "In the garden, I usually"

print("Base GPT-2 Output:")

inputs = tokenizer(prompt, return_tensors="pt")
print(tokenizer.batch_decode(model.generate(**inputs, num_beams=1, do_sample=True, max_new_tokens=40)))


hate_vector = GloVe['hate']

top_k_val = 5
NUM_TOK_TO_GEN = 40

prompt_beams = [prompt] * 4
print(prompt_beams)

for token_num in range(NUM_TOK_TO_GEN):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model(**inputs, labels=inputs["input_ids"])
    #print(outputs)
    loss = outputs.loss
    logits = outputs.logits
    next_token_scores = logits[:, -1, :].softmax(dim=-1)

    sorted_vals, indices = torch.sort(next_token_scores[0])
    sorted_vals = sorted_vals[-top_k_val:]
    indices = indices[-top_k_val:]

    #print([tokenizer.decode(word) for word in indices])
    # one of the generated words is blank (ACTUALLY \n)
    #for word in indices:
    #    if tokenizer.decode(word).strip().lower() == '':
    #        print(f'word: {tokenizer.decode(word)}')
    
    top_embeddings = [GloVe[tokenizer.decode(word).strip().lower()] for word in indices]
    #print(top_embeddings)

    dist_score = [torch.linalg.norm(embed-hate_vector) for embed in top_embeddings]
    #print(dist_score)

    hyper_weight = .5

    pre_rerank = sorted_vals.detach().clone()
    # re-rank the weightings, factor in dist_score
    for i in range(len(sorted_vals)):
        if dist_score[i] == 0:
            # avoid division by 0 (exact match)
            sorted_vals[i] = 0
        else:
            sorted_vals[i] += (1/dist_score[i])*hyper_weight

    sort_indices = torch.argsort(sorted_vals)
    final_ranked_indices = [indices[s] for s in sort_indices]
    
    print(f'Generation {token_num}')
    print("Original: ")
    for idx in range(1, top_k_val+1):
        #print(tokenizer.decode(indices[-idx]))
        print(f'{pre_rerank[-idx]:5f} | {tokenizer.decode(indices[-idx]):8s}')

    print("\nAfter Re-rank:")
    s_vals = sorted_vals[sort_indices]
    for idx in range(1, len(final_ranked_indices)+1):
        #print(s_vals)
        #print(idx)
        print(f'{s_vals[-idx]:5f} | {tokenizer.decode(final_ranked_indices[-idx]):8s}')
    print('\n')
    # add some variability here (so that we don't always take the best word)
    best_word = tokenizer.decode(final_ranked_indices[-1])
    prompt += best_word
    print(f'Chose \'{best_word.strip()}\' to get: {prompt}')
    print('-----------------------------\n')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Base GPT-2 Output:
['In the garden, I usually use a bamboo stalk and a straw. The straw was nice and soft, but the soil was not very thick. The plants made a strong and pleasant scent for me when I did these plants. I']
['In the garden, I usually', 'In the garden, I usually', 'In the garden, I usually', 'In the garden, I usually']
Generation 0
Original: 
0.076539 |  have   
0.035736 |  use    
0.035068 |  get    
0.030946 |  take   
0.029044 |  find   

After Re-rank:
0.154306 |  have   
0.126254 |  get    
0.118793 |  use    
0.116057 |  take   
0.115720 |  find   


Chose 'have' to get: In the garden, I usually have
-----------------------------

Generation 1
Original: 
0.289871 |  a      
0.080601 |  to     
0.044188 |  two    
0.039664 |  my     
0.036944 |  the    

After Re-rank:
0.364159 |  a      
0.157017 |  to     
0.118467 |  my     
0.115017 |  two    
0.114169 |  the    


Chose 'a' to get: In the garden, I usually have a
-----------------------------

Generation 2
Origina

KeyError: ''

In [172]:
# old stuff, using 'wte' matrix for the embeddings

prompt = "In the garden, I usually"

embeddings = model.transformer.wte.weight   # replace this matrix with the matrix we make

# currently only supports cluster size of 1 (no looping/averaging)
cluster = "hate"
cluster_tokens = tokenizer(cluster, return_tensors="pt")['input_ids'][0]
cluster_embedding = embeddings[cluster_tokens]# if we had multiple members, would have to average here

top_k_val = 5  # use top-p instead

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits
#hidden_states = outputs.last_hidden_state
#print(hidden_state)
next_token_scores = logits[:, -1, :].softmax(dim=-1)
# change values in next_token_scores[0] and then torch.sort to get indices
#print(next_token_scores.shape)



sorted_vals, indices = torch.sort(next_token_scores[0])
# generate embedding for each index position
# have 3 vectors, sorted_vals, indices, and embeddings. 
# turn embedding column into distance to cluster column
# merge sorted_vals and embeddings columns (re-weight)
# re-sort indices according to sorted_vals weights
sorted_vals = sorted_vals[-top_k_val:]
indices = indices[-top_k_val:]
#print(indices)
top_embeddings = embeddings[indices]
#print(top_embeddings[0])
#print(cluster_embedding)

dist_score = [torch.linalg.norm(embed-cluster_embedding) for embed in top_embeddings]

hyper_weight = .5

checkpoint = sorted_vals.detach().clone()
for i in range(len(sorted_vals)):
    sorted_vals[i] += (1/dist_score[i])*hyper_weight

sort_indices = torch.argsort(sorted_vals)
final_ranked_indices = [indices[s] for s in sort_indices]
# best result is at the back of indices



#print(sorted_vals)
#print(sort_indices)
print("Original: ")
for idx in range(1, top_k_val+1):
    #print(tokenizer.decode(indices[-idx]))
    print(f'{checkpoint[-idx]:5f} | {tokenizer.decode(indices[-idx]):8s}')

print()
print("After Weighting: ")
s_vals = sorted_vals[sort_indices]
for idx in range(1, len(final_ranked_indices)+1):
    #print(s_vals)
    #print(idx)
    print(f'{s_vals[-idx]:5f} | {tokenizer.decode(final_ranked_indices[-idx]):8s}')
print()
print([tokenizer.decode(word) for word in final_ranked_indices])
print(s_vals)

#next_token = next_token_scores.argmax().unsqueeze(0).unsqueeze(0)
#print(tokenizer.decode(next_token[0]))

Original: 
0.076539 |  have   
0.035736 |  use    
0.035068 |  get    
0.030946 |  take   
0.029044 |  find   

After Weighting: 
0.187514 |  have   
0.145204 |  get    
0.139645 |  take   
0.139214 |  use    
0.138316 |  find   

[' find', ' use', ' take', ' get', ' have']
tensor([0.1383, 0.1392, 0.1396, 0.1452, 0.1875], grad_fn=<IndexBackward0>)
